In [5]:
import pyabf  # import the package
abf = pyabf.ABF("data/2025_12_04_0001.abf")  # load the file
abf.setSweep(1)  # Select the sweep number
abf.sweepY       # sweep data
abf.sweepX       # time data
abf.sweepC       # command waveform

ModuleNotFoundError: No module named 'pyabf'